<a href="https://colab.research.google.com/github/lamyse1/Biomed-Info./blob/main/Graded_Assignment1_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Precision-cancer

Packages:
Numpy version: 1.20.0;
Pandas version: 1.2.1;
scikit-learn version: 0.24.1;
lifelines version: 0.26.0

In [1]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 11.2 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=eae033d7b552e7f3b84511fc016529740c669a68d1cec12b2b4c1058bdde11e1
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [2]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter, KaplanMeierFitter
from sklearn.linear_model import LogisticRegression
import warnings
warnings.simplefilter(action='ignore')

# Step1:  loaded all core libraries: pandas, NumPy, lifelines, and scikit- learn, plus a warning filter so the notebook runs quietly.

## Load Synthetic Data

For this notebook, we use synthetic data in "synthetic_data.csv" as a demonstration of our pipeline. It includes the following features -
- Confounders. We note that here is one toy example. In real applications, more confounders and more categories should be included.
    - <font color=darkblue>*Gender*</font>: whether the patient is female (=1) or not (=0).
    - <font color=darkblue>*Age*</font>: whether age >= 40 (=1) or not (=0).
    - <font color=darkblue>*ECOG*</font>: whether ECOG >=3 (=1) or not (=0).
- <font color=darkblue>*Mutation status*</font> for Gene A and Gene B.
    - Whether the gene is mutated (=1) or not (=0).
- <font color=darkblue>*Treatment status*</font> for Treatment 1 and Treatment 2.
    - Whether the patient received the treatment (=1) or not (=0).
- Survival information
    - <font color=darkblue>*duration*</font>: time from start of treatment to event (days).
    - <font color=darkblue>*event*</font>: death happened (=1) or right censored (=0)
    - <font color=darkblue>*fmi_test*</font>: time from start of treatment to FMI test (days).

In [3]:
data_cox = pd.read_csv('synthetic_data.csv')
print('Number of patients: %d' % data_cox.shape[0])
cols_basic = ['duration', 'event', 'fmi_test'] # Basic infomation for survival analysis
confounders = ['Female', 'Age >= 40', 'ECOG >= 3'] # Confounders
data_cox.head()

Number of patients: 3000


,Female,Age >= 40,ECOG >= 3,Gene A,Gene B,Treatment 1,Treatment 2,duration,event,fmi_test
0,1,1,0,0,0,0,1,439,1,31
1,1,1,1,1,0,1,0,258,0,9
2,1,1,1,1,1,0,1,132,1,31
3,0,0,0,0,0,0,1,62,1,28
4,0,0,0,0,0,0,1,46,1,32


# Step:2 we loaded the synthetic dataset from “synthetic_data.csv,” confirmed there were 3000 patients, defined the basic survival columns (duration,event fmi_test) and the confounders (Female Age>=40 ECOG>=3), and displayed the first few rows.

## Analysis

In [4]:
def process_HR(cph_summary, name_feature):
    '''
    Process result of CoxPH model to HR with 95% confidence intervals.
    '''
    result = cph_summary.loc[name_feature, ['exp(coef)', 'exp(coef) lower 95%', 'exp(coef) upper 95%']].values
    HR = '%.2f (%.2f, %.2f)' % tuple(result)
    return HR

def Cox_IPTW(data_fit, name_feature):
    '''
    Uni-covariate CoxPH model with IPTW and left-truncation.
    name_feature is the feature we are interested in (e.g., gene name or treatment name).
    Return the HR for name_feature.
    '''
    # Generate data used in Cox Regression with IPTW Weights
    data_confounders = data_fit.iloc[:, 3:].copy().drop(columns=[name_feature])
    df = pd.concat([data_fit.iloc[:, :3], data_fit[[name_feature]]], axis=1)
    X = data_confounders.values
    y = data_fit[name_feature].values
    model = LogisticRegression(solver='lbfgs', n_jobs=-1, class_weight='balanced')
    model.fit(X, y)
    p_treated = float(np.sum(y==1))/y.shape[0]
    propensity_scores = model.predict_proba(X)[:, 1]
    df['weights'] = 0
    IP_treated = 1 / propensity_scores
    IP_untreated = 1 / (1 - propensity_scores)
    df.loc[df[name_feature]==1, 'weights'] = IP_treated[df[name_feature]==1]
    df.loc[df[name_feature]==0, 'weights'] = IP_untreated[df[name_feature]==0]
    # Cox Regression
    cph = CoxPHFitter()
    cph.fit(df, 'duration', 'event', weights_col='weights', entry_col='fmi_test', robust=True)
    HR = process_HR(cph.summary, name_feature)
    return HR

# Step3 : defined two helper functions. The first, process_HR, extracted a feature’s hazard ratio plus its 95 % confidence bounds from a fitted CoxPH model summary and formatted them as a string. The second, Cox_IPTW, implemented a one‑variable Cox model with inverse‑probability‑of‑treatment weighting and left‑truncation: it pulled out confounders, fit a logistic regression to estimate propensity scores, computed weights, ran the weighted CoxPH fit on duration/event with entry times, and then used process_HR to return the feature’s hazard ratio.


### 1. Association between mutations and patient survival.

Here we study the prognostic effects of mutations in Gene A on overall survival (OS) and report the OS HR.

In [5]:
data_fit = data_cox[cols_basic+confounders+['Gene A']]
HR = Cox_IPTW(data_fit, 'Gene A')
print('OS HR for Gene A: %s' % (HR))

OS HR for Gene A: 1.09 (1.01, 1.19)


# Step 4 : we ran the prognostic analysis by subsetting the data to include duration event fmi_test the confounders and Gene A then called Cox_IPTW which returned an overall survival hazard ratio of 1.09 (1.01, 1.19) for Gene A mutations.The estimated hazard ratio of 1.09 (95 % CI 1.01–1.19) indicates that patients with the Gene A mutation had a 9 % higher instantaneous risk of death compared to those without the mutation, after adjusting for age, sex, performance status, and accounting for delayed entry. Because the confidence interval does not cross 1, this suggests the mutation is a statistically significant adverse prognostic factor.

### 2. Gene-treatment interactions

Here we use analyze the interaction between mutations of Gene A and Treatment 1.

In [6]:
gene = 'Gene A'
treatment = 'Treatment 1'

##### Interaction

Interaction: exp(coef) for the drug-treatment interaction term in the Cox model, adjusted by patient confounders.

In [7]:
data_fit = data_cox[cols_basic+[gene, treatment]]
col_inter = '%s+%s'%(gene, treatment)
data_fit[col_inter] = data_fit[gene]*data_fit[treatment]
cph = CoxPHFitter()
cph.fit(data_fit, 'duration', 'event', entry_col='fmi_test', robust=True)
HR = process_HR(cph.summary, col_inter)
print('Interaction between Gene A and Treatment 1 is %s' % HR)

Interaction between Gene A and Treatment 1 is 0.65 (0.55, 0.77)


##### Gene-centric
- <font color=darkblue>*HR for gene mutation*</font>: for all patients with mutations of Gene A, OS HR for patients who received Treatment 1 vs. patients who did not take Treatment 1.
- <font color=darkblue>*HR for absence of gene mutation*</font>: for all patients *without* mutations of Gene A, OS HR for patients who received Treatment 1 vs. patients who did not take Treatment 1.

In [8]:
data_fit = data_cox.loc[data_cox[gene]==1, cols_basic+confounders+[treatment]].copy()
HR = Cox_IPTW(data_fit, treatment)
print('Treatment HR for patients with mutations of Gene A = %s' % HR)

data_fit = data_cox.loc[data_cox[gene]==0, cols_basic+confounders+[treatment]].copy()
HR = Cox_IPTW(data_fit, treatment)
print('Treatment HR for patients without mutations of Gene A = %s' % HR)

Treatment HR for patients with mutations of Gene A = 0.79 (0.69, 0.89)
Treatment HR for patients without mutations of Gene A = 1.20 (1.08, 1.34)


##### Drug-centric

- <font color=darkblue>*HR given drug*</font>: for all patients who received Treatment 1, HR for patients with mutations of Gene A vs. patients without mutations of Gene A.
- <font color=darkblue>*HR given other drugs*</font>: for all patients who received other treatments, HR for patients with mutations of Gene A vs. patients without mutations of Gene A.

In [9]:
data_fit = data_cox.loc[data_cox[treatment]==1, cols_basic+confounders+[gene]].copy()
HR = Cox_IPTW(data_fit, gene)
print('Gene A mutation HR for patients who received Treatment 1 = %s' % HR)

data_fit = data_cox.loc[data_cox[treatment]==0, cols_basic+confounders+[gene]].copy()
HR = Cox_IPTW(data_fit, gene)
print('Gene A mutation HR for patients who didn\'t receive Treatment 1 = %s' % HR)

Gene A mutation HR for patients who received Treatment 1 = 0.90 (0.81, 1.01)
Gene A mutation HR for patients who didn't receive Treatment 1 = 1.41 (1.24, 1.60)


# Step 5: tested the Gene A × Treatment 1 interaction by creating an interaction term (Gene A * Treatment 1) fitting a Cox model with entry times, and yielded an interaction hazard ratio of 0.65 (0.55–0.77), showing a 35 % synergistic survival benefit when the mutation and drug coincided. Step 6 then broke this down: among Gene A carriers, Treatment 1 cut mortality by 21 % (HR 0.79 (0.69–0.89)); among non‑carriers, the same drug increased risk by 20 % (HR 1.20 (1.08–1.34)). Finally, in drug‑centric analyses, carriers of Gene A treated with the drug fared slightly better (HR 0.90 (0.81–1.01)), whereas untreated carriers fared much worse (HR 1.41 (1.24–1.60)), confirming genuine effect modification.

### 3. Mutation-mutation interactions

Here we study the mutation-mutation interactions that modify the effect of anchor Gene A.
For patients with anchor gene (Gene A) positive, we extracted the OS HR by comparing cohorts with modifier gene (Gene B) mutation-positive vs. modifier gene (Gene B) mutation-negative.

In [10]:
gene_anchor = 'Gene A'
gene_modifier = 'Gene B'
data_fit = data_cox.loc[data_cox[gene_anchor]==1, cols_basic+confounders+[gene_modifier]]
HR = Cox_IPTW(data_fit, gene_modifier)
print('modifier gene HR for anchor gene positive: %s' % HR)

modifier gene HR for anchor gene positive: 0.85 (0.75, 0.97)


# Step6:  examined mutation–mutation synergy by selecting only Gene A‑positive patients, comparing those with and without Gene B mutations, and found a hazard ratio of 0.85 (0.75, 0.97), indicating that in Gene A carriers the additional Gene B mutation conferred a 15 % reduction in risk.

# So in brief explannation, in this notebook we first imported the necessary libraries, then loaded and validated the synthetic dataset, defined helper functions to compute inverse‑probability‑weighted Cox models and format hazard ratios, assessed the prognostic impact of Gene A (HR 1.09), quantified the synergistic effect between Gene A and Treatment 1 (interaction HR 0.65) and its stratified effects in both mutation‑centric and drug‑centric views (HRs ranging from 0.79 to 1.41), and finally demonstrated that adding Gene B in the Gene A‑positive subgroup further reduced risk (HR 0.85), thus recreating the full analytical pipeline of Liu et al. in a step‑by‑step, reproducible manner.